In [1]:
# IMPORTING MODULES:
import pickle 
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

#@ IGNORING WARNINGS: 
import warnings
warnings.filterwarnings("ignore")

In [7]:
# IMPORTING AND SETTING UP MLFLOW:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [3]:
# OVERALL FUNCTION TO PREPARE DATAFRAME:
def read_dataframe(filename):                                                           # Defining function.
    if filename.endswith(".csv"):                                                       # Checking.
        df = pd.read_csv(filename)                                                      # Reading the dataset.
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)             # Converting to datetime. 
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)               # Converting to datetime. 
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)
        
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)                                       # Conversion.
    return df

def prepare_dictionaries(df: pd.DataFrame):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"] 
    dicts = df[categorical + numerical].to_dict(orient="records")   # Initializing dictionary.
    return dicts

In [5]:
# LOADING THE DATASET:
df_train = read_dataframe("data/green_tripdata_2021-01.parquet")           # Training dataframe.
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")             # Validation dataframe.

# PREPARING THE DATAFRAME:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)  # Preparing dictionaries.
dict_val = prepare_dictionaries(df_val)      # Preparing dictionaries.

In [11]:
# RANDOM FOREST REGRESSOR:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10,
                  random_state=2022)
    mlflow.log_params(params)
    
    dv = DictVectorizer()
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )                                                       # Initializing pipeline.
    
    pipeline.fit(dict_train, y_train)                       # Training the pipeline.
    y_pred = pipeline.predict(dict_val)                     # Initializing prediction.
    rmse = mean_squared_error(y_pred, y_val, squared=False) # Inspecting MSE.
    
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    
    with open("dict_vectorizer.bin", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("dict_vectorizer.bin")

In [15]:
# INTERACTING WITH MODEL REGISTRY:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
run_id = "e46fafe797f94d07a522ad82dc1b5af8"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [16]:
# DOWNLOADING ARTIFACTS:
path = client.download_artifacts(run_id=run_id, path="dict_vectorizer.bin")

In [17]:
# INITIALIZATION:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)
dv

DictVectorizer()